In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import librosa

from tqdm import tqdm_notebook as tqdm

from recording import Recording
from auxiliary import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(font='serif')

import os,string

from collections import Counter

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import mutual_info_classif

In [ ]:
dh = DataHandler()

In [ ]:
df = dh.build_feature_data(condition=2)

In [ ]:
valid_phonemes = list(''.join([string.ascii_letters,string.digits,'@']))
df = df[df.phoneme.isin(valid_phonemes)]
common_phonemes = {phoneme: count for phoneme,count in Counter(df.phoneme).most_common(48)}
common_phonemes

In [ ]:
X = np.squeeze(np.stack(df.feature_vec.tolist()))
y = df.phoneme.tolist()
X.shape

In [ ]:
X_scl = StandardScaler().fit_transform(X)

In [ ]:
pca = PCA(n_components=30)
X_pca = pca.fit_transform(X_scl)
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
fig, ax = plt.subplots(nrows=3,figsize=(12,8))

X_imp = X_pca

feature_score = mutual_info_classif(X_imp,df.phoneme.tolist())
sns.barplot(x=[i+1 for i in range(len(feature_score))],y=feature_score,ax=ax[0])
ax[0].set_title('Feature Importance - Phoneme')

feature_score = mutual_info_classif(X_imp,df.emotion_name.tolist())
sns.barplot(x=[i+1 for i in range(len(feature_score))],y=feature_score,ax=ax[1])
ax[1].set_title('Feature Importance - Emotion')

feature_score = mutual_info_classif(X_imp,df.speaker_id.tolist())
sns.barplot(x=[i+1 for i in range(len(feature_score))],y=feature_score,ax=ax[2])
ax[2].set_title('Feature Importance - Gender')

plt.tight_layout()

In [ ]:
tsne = TSNE(verbose=1)
transformed = tsne.fit_transform(X_pca)

In [ ]:
plot_df = pd.DataFrame({'t-SNE 1': transformed[:,0],
                        't-SNE 2': transformed[:,1],
                        'phoneme': df.phoneme.tolist(), 
                        'emotion': df.emotion_name.tolist(),
                        'speaker': ['s%s (%s)'%(row.speaker_id,row.sex) for i,row in df.iterrows()],
                        'sex': df.sex.tolist(),
                        'pitch': [row.feature_vec[-2] for i,row in df.iterrows()]})

def plot_tSNE_by_hue(data,hue,ax):
    sns.scatterplot('t-SNE 1',
                    't-SNE 2',
                    data=data,
                    hue=hue,
                    hue_order=sorted(set(data[hue])),
                    marker='.',
                    edgecolor=None,
                    alpha=.5,
                    ax=ax)
    ax.legend(bbox_to_anchor=(1, 1))
    ax.set_xlim([int(min(data['t-SNE 1'])-5),int(max(data['t-SNE 1']))+5])
    ax.set_ylim([int(min(data['t-SNE 2'])-5),int(max(data['t-SNE 2']))+5])
    
def visualize_tSNE(data,title):
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(14,12),dpi=150)
    fig.suptitle(title,y=1.02)
    plot_tSNE_by_hue(data,hue='emotion',ax=ax[0][0])
    plot_tSNE_by_hue(data,hue='phoneme',ax=ax[0][1])
    plot_tSNE_by_hue(data,hue='sex',ax=ax[1][0])
    plot_tSNE_by_hue(data,hue='speaker',ax=ax[1][1])
    plt.tight_layout()

phoneme_list = list(common_phonemes.keys())[:6]
visualize_tSNE(plot_df[plot_df.phoneme.isin(phoneme_list)],title='t-SNE (Fitted on all phonemes)')